###### Imports and Settings

In [3]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# This notebook outlines the download and formatting process for the University of Wisconsin's Population Health Institute's County Health Rankings for counties places in the GNRC operating region.  

Go to this page: https://www.countyhealthrankings.org/health-data/tennessee?year=2024  
Download the "(YEAR) Tennessee Data" excel workbook  
If the new version is the same as the 2022 download, then it will contain the following pages:  
+ Introduction, Outcomes & Factors Rankings, Outcomes & Factors SubRankings, Ranked Measure Data, Additional Measure Data, Ranked Measure Sources & Years, Additional Measure Sources & Years  
+ For both "Outcomes & Factors Rankings", "Outcomes & Factors SubRankings", and Additional Measure Data": unmerge headers and keep the FIPS, rename the County to "NAME", delete the State column, and keep only the ranks (we won't use the z-scores), and make sure the column headers are consistent with the outcomes or factors. Ensure the counties are alphabetized and move all of the rankings into one sheet.
  

Save these csvs as they come in the Data Downloads folder of Parent Data Gathering  


# I've got 2019 - 2023 formatted this way, then in 2024 they stopped ranking. This is the rankings up to 2023.


## Merge Multiple Years

In [4]:
conn = sq.connect('../Outputs/CountyHealthRankings.db')
#2024 custom
sql_query = pd.read_sql('SELECT * FROM [RankingAdj_2024]', conn)
df = pd.DataFrame(sql_query)

In [5]:
df.head()

,NAME,Year,Health Outcomes Ranking,Health Factors Ranking
0,Williamson,2024,1,1
1,Wilson,2024,2,2
2,Washington,2024,14,3
3,Hamilton,2024,10,4
4,Knox,2024,9,5


In [6]:
colstt = ['Health Outcomes Ranking', 'Health Factors Ranking']

In [7]:
df.set_index(['NAME', 'Year'], inplace = True)
df = df[colstt]

In [8]:
data = pd.read_csv('../Data Downloads/CountyHealthRankings_2023.csv', dtype = str)
data.set_index(['NAME', 'Year'], inplace = True)
data.head(3)

,,Health Outcomes Ranking,Health Factors Ranking,Length of Life Ranking,Quality of Life Ranking,Health Behaviors Ranking,Clinical Care Ranking,Social & Economic Factors Ranking,Physical Environment Ranking,Life Expectancy,Life Expectancy (AIAN),Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,Age-Adjusted Death Rate,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Child Deaths,Child Mortality Rate,Child Mortality Rate (AIAN),Child Mortality Rate (Asian),Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Infant Deaths,Infant Mortality Rate,Infant Mortality Rate (AIAN),Infant Mortality Rate (Asian),Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access to Healthy Foods,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (White),% Insufficient Sleep,# Uninsured Adults,% Uninsured Adults,# Uninsured Children,% Uninsured Children,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,Cohort Size,High School Graduation Rate,% Disconnected Youth,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests,Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Asian),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Black),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on English Language Arts Standardized Tests (White),Average Grade Performance for 3rd Graders on Math Standardized Tests,Average Grade Performance for 3rd Graders on Math Standardized Tests (Asian),Average Grade Performance for 3rd Graders on Math Standardized Tests (Black),Average Grade Performance for 3rd Graders on Math Standardized Tests (Hispanic),Average Grade Performance for 3rd Graders on Math Standardized Tests (White),Segregation Index,Spending per Pupil,School Funding Adequacy,Women's Median Earnings,Men's Median Earnings,Gender Pay Gap,% Enrolled in Free or Reduced Lunch,% Household Income Required for Child Care Expenses,# Child Care Centers,"Child Care Centers per 1,000 Children",# Suicide Deaths,Suicide Rate (Age-Adjusted),Crude Suicide Rate,Suicide Rate (AIAN),Suicide Rate (Asian),Suicide Rate (Black),Suicide Rate (Hispanic),Suicide Rate (White),# Firearm Fatalities,Firearm Fatalities Rate,Firearm Fatalities Rate (AIAN),Firearm Fatalities Rate (Asian),Firearm Fatalities Rate (Black),Firearm Fatalities Rate (Hispanic),Firearm Fatalities Rate (White),# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,MV Mortality Rate (AIAN),MV Mortality Rate (Asian),MV Mortality Rate (Black),MV Mortality Rate (Hispanic),MV Mortality Rate (White),Non-Petitioned Cases,Petitioned Cases,Denominator,Juvenile Arrest Rate,% Voter Turnout,% Census Participation,Traffic Volume,Premature Deaths,Years of Potential Life Lost Rate,YPLL Rate (AIAN),YPLL Rate (Asian),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% LBW (AIAN),% LBW (Asian),% LBW (Black),% LBW (Hispanic),% LBW (White),% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,# Alcohol-Impaired Driving Deaths,# Driving Deaths,% Driving Deaths with Alcohol Involvement,# Chlamydia Cases,Chlamydia Rate,Tee

In [9]:
data = data.filter(colstt, axis = 1)

In [10]:
dfs = [data, df]
data = pd.concat(dfs)
data.head()

,,Health Outcomes Ranking,Health Factors Ranking
NAME,Year,,
Anderson,2023,38,16
Bedford,2023,37,39
Benton,2023,83,80
Bledsoe,2023,15,91
Blount,2023,8,8


In [11]:
data.tail()

,,Health Outcomes Ranking,Health Factors Ranking
NAME,Year,,
Bledsoe,2024,27,91
Lauderdale,2024,73,92
Perry,2024,45,93
Hancock,2024,94,94
Lake,2024,95,95


In [12]:
#import and drop FIPS on the import as we'll join it with the custom module
data = pd.read_csv('../Data Downloads/CountyHealthRankings_2022.csv', dtype = str)
data.set_index(['NAME', 'Year'], inplace = True)

In [13]:
data = data.filter(colstt, axis = 1)

In [14]:
dfs = [data, df]
data = pd.concat(dfs)
data.head()

,,Health Outcomes Ranking,Health Factors Ranking
NAME,Year,,
Anderson,2022,38,14
Bedford,2022,42,62
Benton,2022,86,80
Bledsoe,2022,21,90
Blount,2022,8,7


In [15]:
twentyone = pd.read_csv('../Data Downloads/CountyHealthRankings_2021.csv', dtype = str)
twentyone.set_index(['NAME', 'Year'], inplace = True)

In [16]:
twentyone = twentyone.filter(colstt, axis = 1)

In [17]:
dfs = [data, twentyone]
data = pd.concat(dfs)
data.head(2)

,,Health Outcomes Ranking,Health Factors Ranking
NAME,Year,,
Anderson,2022,38,14
Bedford,2022,42,62


In [18]:
twenty = pd.read_csv('../Data Downloads/CountyHealthRankings_2020.csv', dtype = str)
twenty.set_index(['NAME', 'Year'], inplace = True)

In [19]:
twenty = twenty.filter(colstt, axis = 1)

In [20]:
dfs = [data, twenty]
data = pd.concat(dfs)
data.tail(2)

,,Health Outcomes Ranking,Health Factors Ranking
NAME,Year,,
Williamson,2020,1,1
Wilson,2020,3,2


In [21]:
nineteen = pd.read_csv('../Data Downloads/CountyHealthRankings_2019.csv', dtype = str)
nineteen.set_index(['NAME', 'Year'], inplace = True)

In [22]:
nineteen = nineteen.filter(colstt, axis = 1)

In [23]:
dfs = [data, nineteen]
data = pd.concat(dfs)
data.tail(2)

,,Health Outcomes Ranking,Health Factors Ranking
NAME,Year,,
Williamson,2019,1,1
Wilson,2019,2,2


In [24]:
data = data.reset_index(drop = False)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   NAME                     479 non-null    object
 1   Year                     479 non-null    object
 2   Health Outcomes Ranking  475 non-null    object
 3   Health Factors Ranking   475 non-null    object
dtypes: object(4)
memory usage: 15.1+ KB


In [26]:
data.head()

,NAME,Year,Health Outcomes Ranking,Health Factors Ranking
0,Anderson,2022,38,14
1,Bedford,2022,42,62
2,Benton,2022,86,80
3,Bledsoe,2022,21,90
4,Blount,2022,8,7


In [27]:
#export to the SQLite database
conn = sq.connect('../Outputs/CountyHealthRankings.db')
data.to_sql('Rankings_Annual', conn, if_exists = 'replace', index = False)

479